## Amazon AgentCore Bedrock 코드 인터프리터를 활용한 고급 데이터 분석 - 튜토리얼(스트랜드)
이 튜토리얼에서는 Python을 사용하여 코드 실행을 통해 고급 데이터 분석을 수행하는 AI 에이전트를 만드는 방법을 보여줍니다. LLM에서 생성된 코드를 실행하기 위해 Amazon Bedrock AgentCore 코드 인터프리터를 사용합니다.

이 튜토리얼에서는 AgentCore Bedrock Code Interpreter를 사용하여 다음 작업을 수행하는 방법을 보여줍니다.
1. 샌드박스 환경 설정
2. 사용자 쿼리를 기반으로 코드를 생성하여 고급 데이터 분석을 수행하는 스트랜드 기반 에이전트 구성
3. 코드 인터프리터를 사용하여 샌드박스 환경에서 코드 실행
4. 결과를 사용자에게 표시

## 필수 조건
- Bedrock AgentCore Code Interpreter 액세스 권한이 있는 AWS 계정
- 코드 인터프리터 리소스를 생성하고 관리하는 데 필요한 IAM 권한이 있어야 합니다.
- 필요한 Python 패키지(boto3, bedrock-agentcore 및 strands 포함)가 설치되어 있어야 합니다.
- IAM 역할에 Amazon Bedrock에서 모델을 호출할 수 있는 권한이 있어야 합니다.
- 미국 오리건(us-west-2) 리전의 Claude Sonnet 3.7 및 Claude Sonnet 3.5 모델에 대한 액세스 권한이 있어야 합니다.

## IAM 실행 역할에 다음 IAM 정책이 연결되어 있어야 합니다.

~~~ {
"Version": "2012-10-17",
"Statement": [
    {
        "Effect": "Allow",
        "Action": [
            "bedrock-agentcore:CreateCodeInterpreter",
            "bedrock-agentcore:StartCodeInterpreterSession",
            "bedrock-agentcore:InvokeCodeInterpreter",
            "bedrock-agentcore:StopCodeInterpreterSession",
            "bedrock-agentcore:DeleteCodeInterpreter",
            "bedrock-agentcore:ListCodeInterpreters",
            "bedrock-agentcore:GetCodeInterpreter"
        ],
        "Resource": "*"
    },
    {
        "Effect": "Allow",
        "Action": [
            "logs:CreateLogGroup",
            "logs:CreateLogStream",
            "logs:PutLogEvents"
        ],
        "Resource": "arn:aws:logs:*:*:log-group:/aws/bedrock-agentcore/code-interpreter*"
    }
]
}

## 작동 방식

코드 실행 샌드박스는 코드 인터프리터, 셸, 파일 시스템을 갖춘 격리된 환경을 생성하여 에이전트가 사용자 쿼리를 안전하게 처리할 수 있도록 합니다. 대규모 언어 모델(LLM)을 통해 도구 선택을 돕고, 코드는 이 세션 내에서 실행된 후 합성을 위해 사용자 또는 에이전트에게 반환됩니다.

![architecture local](code-interpreter.png)

## 1. 환경 설정

먼저, 필요한 라이브러리를 가져오고 코드 인터프리터 클라이언트를 초기화해 보겠습니다.

기본 세션 시간 초과는 900초(15분)입니다. 하지만 데이터에 대한 상세 분석을 수행할 것이므로 세션 시작 시 1200초(20분)의 짧은 시간 초과를 설정합니다.

In [ ]:
!pip install --upgrade -r requirements.txt

In [ ]:
from bedrock_agentcore.tools.code_interpreter_client import CodeInterpreter
from strands import Agent, tool
from strands.models import BedrockModel
import json
import pandas as pd
from typing import Dict, Any, List

# Initialize the Code Interpreter within a supported AWS region.
code_client = CodeInterpreter('us-west-2')
code_client.start(session_timeout_seconds=1200)

## 2. 로컬 데이터 파일 읽기

이제 샘플 데이터 파일의 내용을 읽어 보겠습니다. 이 파일은 이름, 선호하는 도시, 선호하는 동물, 선호하는 사물의 4개 열과 약 30만 개의 레코드로 구성된 무작위 데이터로 구성되어 있습니다.

이후 에이전트를 사용하여 이 파일을 분석하여 분포와 이상치를 파악하겠습니다.

In [ ]:
df_data = pd.read_csv("samples/data.csv")
df_data.head()

In [ ]:
def read_file(file_path: str) -> str:
    """Helper function to read file content with error handling"""
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found.")
        return ""
    except Exception as e:
        print(f"An error occurred: {e}")
        return ""

data_file_content = read_file("samples/data.csv")

## 3. 샌드박스 환경을 위한 파일 준비

샌드박스 환경에 생성할 파일을 정의하는 구조체를 생성합니다.

In [ ]:
files_to_create = [
                {
                    "path": "data.csv",
                    "text": data_file_content
                }]

## 4. 도구 호출을 위한 도우미 함수 생성

이 도우미 함수를 사용하면 샌드박스 도구를 더 쉽게 호출하고 응답을 처리할 수 있습니다. 활성 세션 내에서 지원되는 언어(Python, JavaScript)로 코드를 실행하고, 종속성 구성에 따라 라이브러리에 액세스하고, 시각화를 생성하고, 실행 간 상태를 유지할 수 있습니다.

In [ ]:
def call_tool(tool_name: str, arguments: Dict[str, Any]) -> Dict[str, Any]:
    """Helper function to invoke sandbox tools

    Args:
        tool_name (str): Name of the tool to invoke
        arguments (Dict[str, Any]): Arguments to pass to the tool

    Returns:
        Dict[str, Any]: JSON formatted result
    """
    response = code_client.invoke(tool_name, arguments)
    for event in response["stream"]:
        return json.dumps(event["result"])

## 5. 코드 샌드박스에 데이터 파일 쓰기

이제 샌드박스 환경에 데이터 파일을 쓰고 성공적으로 생성되었는지 확인해 보겠습니다.

In [ ]:
# Write files to sandbox
writing_files = call_tool("writeFiles", {"content": files_to_create})
print("Writing files result:")
print(writing_files)

# Verify files were created
listing_files = call_tool("listFiles", {"path": ""})
print("\nFiles in sandbox:")
print(listing_files)

## 6. 스트랜드 기반 에이전트를 사용하여 고급 분석 수행

이제 샌드박스에 업로드한 데이터 파일(위)에 대한 데이터 분석을 수행하는 에이전트를 구성하겠습니다.

### 6.1 시스템 프롬프트 정의
AI 비서의 동작과 기능을 정의합니다. AI 비서가 코드 실행 및 데이터 기반 추론을 통해 항상 답변을 검증하도록 지시합니다.

In [ ]:
SYSTEM_PROMPT = """귀하는 제공된 도구를 사용하여 코드 실행을 통해 모든 답변을 검증하는 유용한 AI 도우미입니다. 도구를 사용하지 않고 질문에 답변하지 마세요.

검증 원칙:
1. 코드, 알고리즘 또는 계산에 대한 주장을 할 때는 이를 검증하는 코드를 작성하세요.
2. execute_python을 사용하여 수학적 계산, 알고리즘 및 논리를 테스트하세요.
3. 답변하기 전에 이해도를 검증하기 위한 테스트 스크립트를 작성하세요.
4. 항상 실제 코드 실행을 통해 작업을 보여주세요.
5. 확실하지 않은 경우, 제약 사항을 명시적으로 명시하고 검증 가능한 부분을 검증하세요.

접근 방식:
- 프로그래밍 개념에 대한 질문이 있는 경우, 코드로 구현하여 시연하세요.
- 계산을 요청받은 경우, 프로그래밍 방식으로 계산하고 코드를 보여주세요.
- 알고리즘을 구현하는 경우, 정확성을 증명하는 테스트 사례를 포함하세요.
- 투명성을 위해 검증 프로세스를 문서화하세요.
- 샌드박스는 실행 간에 상태를 유지하므로 이전 결과를 참조할 수 있습니다.

사용 가능한 도구:
- execute_python: Python 코드를 실행하고 출력을 확인하세요.

응답 형식: execute_python 도구는 다음을 포함하는 JSON 응답을 반환합니다.
- sessionId: 샌드박스 세션 ID
- id: 요청 ID
- isError: 오류 발생 여부를 나타내는 부울 값
- content: 유형과 텍스트/데이터를 포함하는 콘텐츠 객체 배열
- structuredContent: 코드 실행을 위한 값으로, stdout, stderr, exitCode, executionTime을 포함합니다.

코드가 성공적으로 실행되면 출력은 content[0].text와 structuredContent.stdout에 저장됩니다.
isError 필드를 확인하여 오류가 발생했는지 확인하세요.

답변은 꼼꼼하고 정확하게 작성하며, 가능하면 항상 검증하세요."""

### 6.2 코드 실행 도구 정의
다음으로, 에이전트가 코드 샌드박스에서 코드를 실행하는 데 사용할 함수를 도구로 정의합니다. @tool 데코레이터를 사용하여 해당 함수에 에이전트용 사용자 지정 도구라는 주석을 추가합니다.

활성 코드 인터프리터 세션 내에서 지원되는 언어(Python, JavaScript)로 코드를 실행하고, 종속성 구성에 따라 라이브러리에 액세스하고, 시각화를 생성하고, 실행 간 상태를 유지할 수 있습니다.

In [ ]:
#Define and configure the code interpreter tool
@tool
def execute_python(code: str, description: str = "") -> str:
    """Execute Python code in the sandbox."""

    if description:
        code = f"# {description}\n{code}"

    #Print generated Code to be executed
    print(f"\n Generated Code: {code}")


    # Call the Invoke method and execute the generated code, within the initialized code interpreter session
    response = code_client.invoke("executeCode", {
        "code": code,
        "language": "python",
        "clearContext": False
    })
    for event in response["stream"]:
        return json.dumps(event["result"])

### 6.3 에이전트 구성
Strands SDK를 사용하여 에이전트를 생성하고 구성합니다. 시스템 프롬프트와 위에서 정의한 도구를 제공하여 코드를 실행합니다.

Claude Sonnet 3.7 모델을 사용하고 [교차 지역 추론(CRIS)](https://docs.aws.amazon.com/bedrock/latest/userguide/cross-region-inference.html) 프로필 ID를 지정합니다.

In [ ]:
model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0"
model= BedrockModel(model_id=model_id)

#configure the strands agent including the model and tool(s)
agent=Agent(
    model=model,
        tools=[execute_python],
        system_prompt=SYSTEM_PROMPT,
        callback_handler=None)

## 7. 에이전트 호출 및 응답 처리
쿼리를 사용하여 에이전트를 호출하고 에이전트의 응답을 처리합니다.

참고: 비동기 실행은 비동기 환경에서 실행해야 합니다.

## 7.1 탐색적 데이터 분석(EDA)을 수행하기 위한 쿼리

코드 샌드박스 환경에서 데이터 파일에 대한 탐색적 데이터 분석을 수행하도록 에이전트에게 지시하는 쿼리부터 시작해 보겠습니다.

In [ ]:
query = "'data.csv' 파일을 불러와 탐색적 데이터 분석(EDA)을 수행해 보세요. 분포와 이상치 값에 대해 알려주세요."

# Invoke the agent asynchcronously and stream the response
response_text = ""
async for event in agent.stream_async(query):
    if "data" in event:
        # Stream text response
        chunk = event["data"]
        response_text += chunk
        print(chunk, end="")

## 7.2 정보 추출 쿼리

이제 코드 샌드박스 환경에서 에이전트에게 데이터 파일에서 특정 정보를 추출하도록 지시해 보겠습니다.

In [ ]:
query = "'data.csv' 파일에서 이름이 'Kimberly'인 사람 중에 가장 좋아하는 동물이 'Crocodile'인 사람은 몇 명이나 됩니까?"

# Invoke the agent asynchcronously and stream the response
response_text = ""
async for event in agent.stream_async(query):
    if "data" in event:
        # Stream text response
        chunk = event["data"]
        response_text += chunk
        print(chunk, end="")

## 8. 정리

마지막으로, 코드 인터프리터 세션을 종료하여 정리합니다. 세션 사용이 끝나면 세션을 종료하여 리소스를 해제하고 불필요한 요금 발생을 방지해야 합니다.

In [ ]:
# Stop the Code Interpreter session
code_client.stop()
print("Code Interpreter session stopped successfully!")